In [93]:
!pip install --upgrade yt_dlp
!pip install pytubefix"

zsh:1: unmatched "


In [94]:
from pytubefix import YouTube
import os

yt = YouTube("https://www.youtube.com/watch?v=sV2t3tW_JTQ")

video = yt.streams.filter(only_audio=True).first()

print("Enter the destination (leave blank for current directory)")
destination = ""

out_file = video.download(output_path=destination)

base, ext = os.path.splitext(out_file)
new_file = "song" + '.wav'
os.rename(out_file, new_file)

print(yt.title + " has been successfully downloaded.")

Enter the destination (leave blank for current directory)
21 Savage - Bank Account (Official Audio) has been successfully downloaded.


In [7]:
import requests

In [33]:
# api_url = "https://youtube-to-mp315.p.rapidapi.com/download"
# params = {"url": "https://www.youtube.com/watch?v=sV2t3tW_JTQ"}
# headers = {'x-rapidapi-key': '4ee8496612msh1c77216f8f0acc3p134ca0jsn3ea9d4d86322','x-rapidapi-host': 'youtube-to-mp315.p.rapidapi.com'}

#     # Get MP3 file from API
# r = requests.post(api_url, params=params, headers=headers)

mp = requests.get("http://78.141.232.210/17daea4a-5c61-4041-908a-35defbb13397.mp3")
with open("audio.mp3", "wb") as f:
    f.write(mp.content)

In [64]:
r

<Response [200]>

In [4]:
# !pip install essentia --upgrade

In [5]:
# import essentia.standard as es

# # Load audio

# extractor = es.MusicExtractor()
# features_te, features_frames_te = extractor("song.mp3")

# # Print features
# print(f"Tempo: {features_te['rhythm.bpm']}")
# print(f"Energy: {features_te['lowlevel.average_loudness']}")

In [6]:
# !pip install -v "librosa==0.9"

In [35]:
import os
import multiprocessing
import warnings
import numpy as np
from scipy import stats
import pandas as pd
import librosa
from tqdm import tqdm
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1,mel_spec=128)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def compute_features():

    features = pd.Series(index=columns(), dtype=np.float32)

    # Catch warnings as exceptions (audioread leaks file descriptors).
    warnings.filterwarnings('error', module='librosa')

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    try:
        x, sr = librosa.load("audio.mp3", sr=None, mono=True)  # kaiser_fast

        f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
        feature_stats('zcr', f)

        cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))
        
        assert cqt.shape[0] == 7 * 12
        assert np.ceil(len(x)/512) <= cqt.shape[1] <= np.ceil(len(x)/512)+1

        f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cqt', f)
        f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
        feature_stats('chroma_cens', f)
        f = librosa.feature.tonnetz(chroma=f)
        feature_stats('tonnetz', f)

        del cqt
        stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))
        assert stft.shape[0] == 1 + 2048 // 2
        assert np.ceil(len(x)/512) <= stft.shape[1] <= np.ceil(len(x)/512)+1
        del x

        f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
        feature_stats('chroma_stft', f)

        f = librosa.feature.rms(S=stft)
        feature_stats('rmse', f)

        f = librosa.feature.spectral_centroid(S=stft)
        feature_stats('spectral_centroid', f)
        f = librosa.feature.spectral_bandwidth(S=stft)
        feature_stats('spectral_bandwidth', f)
        f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
        feature_stats('spectral_contrast', f)
        f = librosa.feature.spectral_rolloff(S=stft)
        feature_stats('spectral_rolloff', f)
        f = librosa.feature.melspectrogram(S=stft)
        feature_stats('mel_spec', f)
        
        
        
        mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)
        del stft
        f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
        feature_stats('mfcc', f)
        print(f)

    except Exception as e:
        print('{}'.format( repr(e)))

    return features

In [37]:
features = pd.DataFrame(compute_features())

/var/folders/gv/571nb6gs4h58lv0gw0__gy280000gn/T/ipykernel_14135/2343918166.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.02753561]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  features[name, 'mean'] = np.mean(values, axis=1)


[[-463.2535    -463.2535    -463.2535    ... -323.339     -330.01834
  -332.5143   ]
 [   0.           0.           0.        ...  117.43568    115.158966
   115.77475  ]
 [   0.           0.           0.        ...   25.478025    28.567688
    34.99578  ]
 ...
 [   0.           0.           0.        ...  -17.323368   -16.840916
   -20.76704  ]
 [   0.           0.           0.        ...  -21.634237   -20.653526
   -23.273798 ]
 [   0.           0.           0.        ...   -1.469102    -2.0000947
    -1.7020481]]


In [39]:
features

0
feature     statistics number          
chroma_cens kurtosis   01     -0.273852
                       02      0.846372
                       03     -1.267982
                       04     -1.048105
                       05      0.467149
...                                 ...
zcr         mean       01      0.027536
            median     01      0.019043
            min        01      0.000000
            skew       01      2.220871
            std        01      0.029319

[1414 rows x 1 columns]

In [41]:
features = features.transpose()

In [43]:
mel_spec = features['mel_spec']

In [45]:
mel_spec.columns = pd.MultiIndex.from_tuples([
    ('mel_spec',) + col if isinstance(col, tuple) else ('mel_spec', col)
    for col in mel_spec.columns
])

In [47]:
mel_spec

mel_spec                                                                 \
   kurtosis                                                                  
         01        02         03         04        05         06        07   
0 -0.660883  5.006852  15.681183  10.449755  9.435921  15.799994  7.470451   

                                   ...                                         \
                                   ...       std                                
          08         09        10  ...        90        91       92        93   
0  11.867121  10.998114  9.421731  ...  0.002722  0.003452  0.00214  0.001427   

                                                              
                                                              
         94       95        96        97        98        99  
0  0.000163  0.00001  0.000008  0.000008  0.000008  0.000008  

[1 rows x 896 columns]

In [49]:
features = features.drop('mel_spec',axis=1).merge(mel_spec,left_index=True,right_index=True)

In [51]:
features

chroma_cens                                                              \
     kurtosis                                                               
           01        02        03        04        05        06        07   
0   -0.273852  0.846372 -1.267982 -1.048105  0.467149  0.312335  3.789103   

                                 ...  mel_spec                               \
                                 ...       std                                
         08        09        10  ...        90        91       92        93   
0 -0.397305  1.204172  6.673942  ...  0.002722  0.003452  0.00214  0.001427   

                                                              
                                                              
         94       95        96        97        98        99  
0  0.000163  0.00001  0.000008  0.000008  0.000008  0.000008  

[1 rows x 1414 columns]

In [53]:
features.to_csv("bank_acc.csv")


In [370]:
import pickle
with open('top_5_genre.pkl', 'rb') as f:
    model = pickle.load(f)

In [372]:
model.predict(features)

array([15])

In [230]:
y_pred = model.predict(features)
y_pred

array([15])

In [49]:
y_pred_binary_transpose= y_pred_binary.transpose()

In [51]:
max(y_pred_binary_transpose[0])

72.0

In [ ]:
y_pred_binary_transpose[y_pred_binary_transpose[0]==49.0]

,0
18,49.0
